In [5]:
import pandas as pd
from utils import formatar_colunas
import os

In [6]:
diretorio_atual = os.getcwd()
caminho_dados = os.path.join(diretorio_atual, 'dados')

In [7]:
for arquivo in os.listdir(caminho_dados):
    caminho_arquivo = os.path.join(caminho_dados, arquivo)

    if 'Preventiva' in arquivo:
        df_preventiva = pd.read_excel(caminho_arquivo)
        df_preventiva = formatar_colunas(df_preventiva)
        df_preventiva = df_preventiva.add_prefix('Preventiva_')
    elif 'Carreta' in arquivo:
        df_carreta = pd.read_excel(caminho_arquivo)
        df_carreta = formatar_colunas(df_carreta)
        df_carreta = df_carreta.add_prefix('Carreta_')
    elif 'magazine' in arquivo:
        df_esl = pd.read_excel(caminho_arquivo)
        df_esl = formatar_colunas(df_esl)
        df_esl = df_esl.add_prefix('Esl_')
    elif 'Mobile' in arquivo:
        df_mobile = pd.read_excel(caminho_arquivo)
        df_mobile = formatar_colunas(df_mobile)
        df_mobile = df_mobile.add_prefix('Mobile_')
    elif 'Bipe_Produtos' in arquivo:
        df_bipe = pd.read_excel(caminho_arquivo)
        df_bipe = formatar_colunas(df_bipe)
        df_bipe = df_bipe.add_prefix('Bipe_Prod_')
    elif 'Bipe_de_notas' in arquivo:
        df_bipe_notas = pd.read_excel(caminho_arquivo, sheet_name='Plan1')
        df_bipe_notas = formatar_colunas(df_bipe_notas)
        df_bipe_notas = df_bipe_notas.add_prefix('Bipe_Notas_')

In [8]:
df_bipe_notas.columns

Index(['Bipe_Notas_Nf', 'Bipe_Notas_Chave', 'Bipe_Notas_Ocorrencia',
       'Bipe_Notas_Cd'],
      dtype='object')

In [11]:
df_carreta = df_carreta[['Carreta_Pedido', 'Carreta_Nf`S', 'Carreta_Chave', 'Carreta_Data Entrada', 'Carreta_Tipo_Fluxo']]

In [13]:
df_esl.columns

Index(['Esl_Data Do Frete', 'Esl_Remetente/Nome Fantasia',
       'Esl_Endereço Principal/Logradouro', 'Esl_Endereço Principal/Número',
       'Esl_Cidade/Nome', 'Esl_Remetente/Documento', 'Esl_Destinatário/Nome',
       'Esl_Endereço Principal/Logradouro.1',
       'Esl_Endereço Principal/Número.1', 'Esl_Endereço Principal/Complemento',
       'Esl_Endereço Principal/Cep', 'Esl_Endereço Principal/Bairro',
       'Esl_Cidade/Nome.1', 'Esl_Estado/Nome',
       'Esl_Pagador Do Frete/Nome Fantasia', 'Esl_Nota Fiscal/Número',
       'Esl_Nota Fiscal/Série', 'Esl_Nota Fiscal/N° Pedido',
       'Esl_Ultimo Manifesto', 'Esl_Pessoa/Nome', 'Esl_Veículo/Placa',
       'Esl_Manifesto/Número', 'Esl_Manifesto/Data',
       'Esl_Última Ocorrência/Data Ocorrência', 'Esl_Ocorrência/Ocorrência',
       'Esl_Ocorrência/Ocorrência.1',
       'Esl_Última Ocorrência Com Ação/Data Ocorrência',
       'Esl_Manifesto Entrega/N° Tentativas',
       'Esl_Última Ocorrência/Observações', 'Esl_Manifesto/Número.1',

In [14]:
df_bipe_notas.columns

Index(['Bipe_Notas_Nf', 'Bipe_Notas_Chave', 'Bipe_Notas_Ocorrencia',
       'Bipe_Notas_Cd'],
      dtype='object')

In [15]:
df_bipe.columns

Index(['Bipe_Prod_Pedido', 'Bipe_Prod_Status_Deposito'], dtype='object')

In [27]:
df_final = (pd.merge(df_carreta, df_esl[['Esl_Nota Fiscal/Chave Nf-E', 
                                         'Esl_Última Ocorrência/Observações', 'Esl_Pessoa/Nome',
                                           'Esl_Última Ocorrência/Data Ocorrência'  ]], 
                                           left_on='Carreta_Chave', right_on='Esl_Nota Fiscal/Chave Nf-E', how='left')
                                           .drop(columns=['Esl_Nota Fiscal/Chave Nf-E'])
                                           .merge(df_mobile[['Mobile_Pedido', 'Mobile_Tipo', 'Mobile_Entregador']], left_on='Carreta_Pedido', right_on='Mobile_Pedido', how='left').drop(columns=['Mobile_Pedido'])
                                           .merge(df_bipe_notas[['Bipe_Notas_Chave','Bipe_Notas_Ocorrencia']], left_on='Carreta_Chave', right_on='Bipe_Notas_Chave', how='left').drop(columns=['Bipe_Notas_Chave'])
                                           .merge(df_bipe[['Bipe_Prod_Pedido', 'Bipe_Prod_Status_Deposito']], left_on='Carreta_Pedido', right_on='Bipe_Prod_Pedido', how='left').drop(columns=['Bipe_Prod_Pedido'])
                                           
                                           )

In [28]:
df_final_2 = df_final.merge(df_esl[['Esl_Nota Fiscal/Chave Nf-E', 'Esl_Ocorrência/Ocorrência']], left_on='Carreta_Chave', right_on='Esl_Nota Fiscal/Chave Nf-E', how='left').drop(columns=['Esl_Nota Fiscal/Chave Nf-E'])

In [29]:
df_final_2['Esl_Última Ocorrência/Observações'] = df_final_2['Esl_Última Ocorrência/Observações'].fillna('Não informado')

In [30]:
filtro = df_final_2['Esl_Última Ocorrência/Observações'] == 'Não informado'

In [31]:
df_final_2.loc[filtro, 'Esl_Última Ocorrência/Observações'] = df_final_2.loc[filtro, 'Esl_Ocorrência/Ocorrência']


In [32]:
df_final_2 =  df_final_2.drop(columns=['Esl_Ocorrência/Ocorrência'])

In [33]:
df_final_2.to_excel('Relatorio_diario.xlsx', sheet_name='Analise', index=False)